# Run EMR Job via Boto
Objective: Develop code to execute EMR job using Boto alone.  
https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/emr.html

In [1]:
import boto3

In [5]:
client = boto3.session.Session(profile_name='default').client('emr')

In [7]:
client.list_clusters()

{'Clusters': [{'Id': 'j-3NO2D6LP8TVCS',
   'Name': 'emr-kp',
   'Status': {'State': 'TERMINATED',
    'StateChangeReason': {'Code': 'USER_REQUEST',
     'Message': 'Terminated by user request'},
    'Timeline': {'CreationDateTime': datetime.datetime(2023, 4, 19, 6, 42, 28, 698000, tzinfo=tzlocal()),
     'ReadyDateTime': datetime.datetime(2023, 4, 19, 6, 46, 57, 88000, tzinfo=tzlocal()),
     'EndDateTime': datetime.datetime(2023, 4, 19, 6, 52, 36, 573000, tzinfo=tzlocal())}},
   'NormalizedInstanceHours': 8,
   'ClusterArn': 'arn:aws:elasticmapreduce:us-east-1:655268872845:cluster/j-3NO2D6LP8TVCS'}],
 'ResponseMetadata': {'RequestId': '0f047e40-e676-4e2d-bd11-882a6b93949f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '0f047e40-e676-4e2d-bd11-882a6b93949f',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '398',
   'date': 'Tue, 25 Apr 2023 11:07:42 GMT'},
  'RetryAttempts': 0}}

In [7]:
client.describe_cluster(ClusterId='j-3NO2D6LP8TVCS')

{'Cluster': {'Id': 'j-3NO2D6LP8TVCS',
  'Name': 'emr-kp',
  'Status': {'State': 'TERMINATED',
   'StateChangeReason': {'Code': 'USER_REQUEST',
    'Message': 'Terminated by user request'},
   'Timeline': {'CreationDateTime': datetime.datetime(2023, 4, 19, 6, 42, 28, 698000, tzinfo=tzlocal()),
    'ReadyDateTime': datetime.datetime(2023, 4, 19, 6, 46, 57, 88000, tzinfo=tzlocal()),
    'EndDateTime': datetime.datetime(2023, 4, 19, 6, 52, 36, 573000, tzinfo=tzlocal())}},
  'Ec2InstanceAttributes': {'Ec2SubnetId': 'subnet-0179f300743d7583c',
   'RequestedEc2SubnetIds': ['subnet-0179f300743d7583c'],
   'Ec2AvailabilityZone': 'us-east-1d',
   'RequestedEc2AvailabilityZones': [],
   'IamInstanceProfile': 'emr-ec2-role',
   'EmrManagedMasterSecurityGroup': 'sg-0b3081a1938c163a5',
   'EmrManagedSlaveSecurityGroup': 'sg-0abb5aa1d5fad5681',
   'AdditionalMasterSecurityGroups': [],
   'AdditionalSlaveSecurityGroups': []},
  'InstanceCollectionType': 'INSTANCE_GROUP',
  'ReleaseLabel': 'emr-6.10.0'

In [15]:
response = client.run_job_flow(
    Name='emr-cluster-kp-0425-boto',
    ReleaseLabel='emr-6.10.0',
    Instances={
        'InstanceGroups': [
            {
                'Name': 'inst1',
                'Market': 'ON_DEMAND',
                'InstanceRole': 'MASTER',
                'InstanceType': 'm5.xlarge',
                'InstanceCount': 1,
            },
        ],
        'KeepJobFlowAliveWhenNoSteps': True,
        'TerminationProtected': False,
        'Ec2SubnetId': 'subnet-0179f300743d7583c',
        'EmrManagedMasterSecurityGroup': 'sg-0b3081a1938c163a5',
        'EmrManagedSlaveSecurityGroup': 'sg-0b3081a1938c163a5',
        'AdditionalMasterSecurityGroups': [],
        'AdditionalSlaveSecurityGroups': []
    },
    Steps=[
    {
        "Name": "run1",
        "ActionOnFailure": "CANCEL_AND_WAIT",
        "HadoopJarStep": {
            "Jar": "command-runner.jar",
            "Args": ["spark-submit", 
                     "--deploy-mode", 
                     "cluster",
                     "s3://weather-data-kpde/code/02_process_s3_parquet.py"],
        },
    }
    ],
    Applications=[{'Name': 'Spark'}],
    VisibleToAllUsers=True,
    JobFlowRole='emr-ec2-role',
    ServiceRole='emr-role',
    AutoTerminationPolicy={
        'IdleTimeout': 1*60*60
    }
)

In [16]:
response

{'JobFlowId': 'j-LGO98Y54966M',
 'ClusterArn': 'arn:aws:elasticmapreduce:us-east-1:655268872845:cluster/j-LGO98Y54966M',
 'ResponseMetadata': {'RequestId': '7030c1c1-71d4-4d62-b687-a18c54806691',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '7030c1c1-71d4-4d62-b687-a18c54806691',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '116',
   'date': 'Tue, 25 Apr 2023 11:25:10 GMT'},
  'RetryAttempts': 0}}